In [ ]:
import matplotlib.pyplot as plt
sorted_counts = pd.DataFrame([c.split('_')[-2] for c in rad_data.columns if c !='ID']).value_counts()
sorted_counts = pd.DataFrame(sorted_counts, columns=['count']).reset_index()
sorted_counts = sorted_counts.sort_values(0)
display(sorted_counts)
plt.pie(sorted_counts['count'], labels=[i for i in sorted_counts[0]], startangle=0,
        counterclock = False, autopct = '%.1f%%')
plt.savefig(f'img/Rad_feature_ratio.svg', bbox_inches = 'tight')

In [ ]:
label_data = pd.read_csv(labelf)
label_data['ID'] = label_data['ID'].map(lambda x: f"{x}.nii.gz" if not (f"{x}".endswith('.nii.gz') or  f"{x}".endswith('.nii')) else x)
label_data = label_data[['ID', 'group'] + labels]
label_data.head()

In [ ]:
# 删掉ID这一列。
combined_data = pd.merge(rad_data, label_data, on=['ID'], how='inner')
ids = combined_data['ID']
combined_data = combined_data.drop(['ID'], axis=1)
print(combined_data[labels].value_counts())
combined_data

In [ ]:
combined_data.describe()

In [ ]:
from onekey_algo.custom.components.comp1 import normalize_df
data = normalize_df(combined_data, not_norm=labels, group=group_info)
data = data.dropna(axis=1)
data.describe()

In [ ]:
import seaborn as sns
from onekey_algo.custom.components.stats import clinic_stats

stats = clinic_stats(data[data['group'] == 'train'], stats_columns=list(data.columns[0:-2]), label_column=labels[0], 
                     continuous_columns=list(data.columns[0:-2]))
stats

In [ ]:
import matplotlib.pyplot as plt

def map2float(x):
    try:
        return float(str(x)[1:])
    except:
        return 1

stats[['pvalue']] = stats[['pvalue']].applymap(map2float)
stats[['group']] = stats[['feature_name']].applymap(lambda x: x.split('_')[-2])
stats = stats[['feature_name', 'pvalue', 'group']]
g = sns.catplot(x="group", y="pvalue", data=stats, kind="violin")
g.fig.set_size_inches(15,10)
sns.stripplot(x="group", y="pvalue", data=stats, ax=g.ax, color='black')
plt.savefig(f'img/Rad_feature_stats.svg', bbox_inches = 'tight')

In [ ]:
pvalue = 0.05
sel_feature = list(stats[stats['pvalue'] < pvalue]['feature_name']) + labels + [group_info]
data = data[sel_feature]
data

In [ ]:
pearson_corr = data[data['group'] == 'train'][[c for c in data.columns if c not in labels]].corr('pearson')

In [ ]:
from onekey_algo.custom.components.comp1 import select_feature
sel_feature = select_feature(pearson_corr, threshold=0.9, topn=10, verbose=False)
sel_feature = sel_feature + labels + [group_info]
sel_feature

In [ ]:
sel_data = data[sel_feature]
sel_data

In [ ]:
from onekey_algo.custom.components.comp1 import analysis_features
analysis_features(data[sel_feature[:-2]], data[labels[0]], methods=None)

In [ ]:
import numpy as np
import onekey_algo.custom.components as okcomp

n_classes = 2
train_data = sel_data[(sel_data[group_info] == 'train')]
train_ids = ids[train_data.index]
train_data = train_data.reset_index()
train_data = train_data.drop('index', axis=1)
y_data = train_data[labels]
X_data = train_data.drop(labels + [group_info], axis=1)

val_data = sel_data[sel_data[group_info] == 'val']
val_ids = ids[val_data.index]
val_data = val_data.reset_index()
val_data = val_data.drop('index', axis=1)
y_val_data = val_data[labels]
X_val_data = val_data.drop(labels + [group_info], axis=1)

test_data = sel_data[sel_data[group_info] == 'test']
test_ids = ids[test_data.index]
test_data = test_data.reset_index()
test_data = test_data.drop('index', axis=1)
y_test_data = test_data[labels]
X_test_data = test_data.drop(labels + [group_info], axis=1)

y_all_data = sel_data[labels]
X_all_data = sel_data.drop(labels + [group_info], axis=1)

column_names = X_data.columns
print(f"训练集样本数：{X_data.shape}, 内部测试集样本数：{X_val_data.shape}, 外部测试集样本数：{X_test_data.shape}")

In [ ]:
alpha = okcomp.comp1.lasso_cv_coefs(X_data, y_data, column_names=None)
plt.savefig(f'img/Rad_feature_lasso.svg', bbox_inches = 'tight')

In [ ]:
okcomp.comp1.lasso_cv_efficiency(X_data, y_data, points=50)
plt.savefig(f'img/Rad_feature_mse.svg', bbox_inches = 'tight')

In [ ]:
from sklearn import linear_model

models = []
for label in labels:
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(X_data, y_data[label])
    models.append(clf)

In [ ]:
COEF_THRESHOLD = 1e-6 # 筛选的特征阈值
scores = []
selected_features = []
for label, model in zip(labels, models):
    feat_coef = [(feat_name, coef) for feat_name, coef in zip(column_names, model.coef_) 
                 if COEF_THRESHOLD is None or abs(coef) > COEF_THRESHOLD]
    selected_features.append([feat for feat, _ in feat_coef])
    formula = ' '.join([f"{coef:+.6f} * {feat_name}" for feat_name, coef in feat_coef])
    score = f"{label} = {model.intercept_} {'+' if formula[0] != '-' else ''} {formula}"
    scores.append(score)
    
print(scores[0])

In [ ]:
feat_coef = sorted(feat_coef, key=lambda x: x[1])
feat_coef_df = pd.DataFrame(feat_coef, columns=['feature_name', 'Coefficients'])
feat_coef_df.plot(x='feature_name', y='Coefficients', kind='barh')

plt.savefig(f'img/Rad_feature_weights.svg', bbox_inches = 'tight')

In [ ]:
X_data = X_data[selected_features[0]]
X_val_data = X_val_data[selected_features[0]]
X_test_data = X_test_data[selected_features[0]]
X_data.columns

In [ ]:
model_names = []
models = okcomp.comp1.create_clf_model(model_names)
model_names = list(models.keys())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score

#results = okcomp.comp1.get_bst_split(X_data, y_data, models, test_size=0.2, metric_fn=roc_auc_score, n_trails=1, cv=False, random_state=10)
#_, (X_train_sel, X_test_sel, y_train_sel, y_test_sel) = results['results'][results['max_idx']]
#X_train_sel, X_test_sel, y_train_sel, y_test_sel = X_data, X_test_data, y_data, y_test_data
#trails, _ = zip(*results['results'])
#cv_results = pd.DataFrame(trails, columns=model_names)
#sns.boxplot(data=cv_results)
#plt.ylabel('AUC %')
#plt.xlabel('Model Name')
#plt.savefig(f'img/Rad_model_cv.svg', bbox_inches = 'tight')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from onekey_algo.custom.components.delong import calc_95_CI
from onekey_algo.custom.components.metrics import analysis_pred_binary

predictions = [[(model.predict(X_train_sel), model.predict(X_val_data), model.predict(X_test_sel)) 
                for model in target] for label, target in zip(labels, targets)]
pred_scores = [[(model.predict_proba(X_train_sel), model.predict_proba(X_val_data), model.predict_proba(X_test_sel)) 
                for model in target] for label, target in zip(labels, targets)]

metric = []
pred_sel_idx = []
for label, prediction, scores in zip(labels, predictions, pred_scores):
    pred_sel_idx_label = []
    for mname, (train_pred, val_pred, test_pred), (train_score, val_score, test_score) in zip(model_names, prediction, scores):
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_train_sel[label], 
                                                                                              train_score[:, 1])
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"{label}-train"))

        print(f"Length of y_val_data[{label}]: {len(y_val_data[label])}")
        print(f"Length of val_score[:, 1]: {len(val_score[:, 1])}")

        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_val_data[label], 
                                                                                              val_score[:, 1])
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"{label}-val"))

        pred_sel_idx_label.append(np.logical_or(val_score[:, 0] >= thres, val_score[:, 1] >= thres))

        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_test_sel[label], 
                                                                                              test_score[:, 1])
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"{label}-test"))

        pred_sel_idx_label.append(np.logical_or(test_score[:, 0] >= thres, test_score[:, 1] >= thres))        
    
    pred_sel_idx.append(pred_sel_idx_label)
metric = pd.DataFrame(metric, index=None, columns=['model_name', 'Accuracy', 'AUC', '95% CI',
                                                   'Sensitivity', 'Specificity', 
                                                   'PPV', 'NPV', 'Precision', 'Recall', 'F1',
                                                   'Threshold', 'Task'])
metric.to_csv('results/metric_results.csv', index=False)
metric

In [ ]:
import shap
from sklearn import svm

shap_model_1 = svm.SVC(probability=True).fit(X_train_sel, y_train_sel[l])

def svm_predict_proba(data):
    return shap_model_1.predict_proba(data)

explainer1 = shap.KernelExplainer(svm_predict_proba, X_train_sel)
shap_values3 = explainer1.shap_values(X_train_sel)

shap.summary_plot(shap_values3[1], X_train_sel, plot_type="bar")

shap.plots.beeswarm(shap.Explanation(values=shap_values3[1], base_values=explainer1.expected_value[1], data=X_train_sel), max_display=20)


In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 10))
plt.subplot(211)
sns.barplot(x='model_name', y='Accuracy', data=metric, hue='Task')
plt.subplot(212)
sns.lineplot(x='model_name', y='Accuracy', data=metric, hue='Task')
plt.savefig(f'img/Rad_model_acc.svg', bbox_inches = 'tight')

In [ ]:
sel_model = model_names

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
    
        # Plot all ROC curves
        plt.figure(figsize=(8, 8))
        for pred_score, label in zip(pred_scores, labels):
            okcomp.comp1.draw_roc([np.array(y_train_sel[label]), np.array(y_val_data[label]), np.array(y_test_sel[label])], 
                                  pred_score[sel_model_idx], 
                                  labels=['Training Set', 'Internal Tseting Set', 'External Testing Set'], title=f"Model: {sm}")
            plt.savefig(f'img/Rad_model_{sm}_roc.svg', bbox_inches = 'tight')

In [ ]:
for pred_score, label in zip(pred_scores, labels):
    pred_train_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            pred_train_scores.append(pred_score[sel_model_idx][0])  # 0 对应训练集预测
    okcomp.comp1.draw_roc([np.array(y_train_sel[label])] * len(pred_train_scores), 
                          pred_train_scores, 
                          labels=sel_model, title=f"{label} - Model AUC - Train")
    plt.savefig(f'img/model_roc_train_{label}.svg', bbox_inches='tight')
    plt.show()
for pred_score, label in zip(pred_scores, labels):
    pred_val_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            pred_val_scores.append(pred_score[sel_model_idx][1])  
    print(f"Length of y_val_data[{label}]: {len(y_val_data[label])}")
    for i, score in enumerate(pred_val_scores):
        print(f"Length of pred_val_scores[{i}]: {len(score)}")

    if len(y_val_data[label]) == len(pred_val_scores[0]):  
        okcomp.comp1.draw_roc([np.array(y_val_data[label])] * len(pred_val_scores), 
                              pred_val_scores, 
                              labels=sel_model, title=f"{label} - Model AUC - Validation")
        plt.savefig(f'img/model_roc_val_{label}.svg', bbox_inches='tight')
        plt.show()
    else:
        print(f"长度不匹配: y_val_data[{label}] 和 pred_val_scores[0]")

for pred_score, label in zip(pred_scores, labels):
    pred_test_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            pred_test_scores.append(pred_score[sel_model_idx][2]) 
    print(f"Length of y_test_sel[{label}]: {len(y_test_sel[label])}")
    for i, score in enumerate(pred_test_scores):
        print(f"Length of pred_test_scores[{i}]: {len(score)}")

    if len(y_test_sel[label]) == len(pred_test_scores[0]):
        okcomp.comp1.draw_roc([np.array(y_test_sel[label])] * len(pred_test_scores), 
                              pred_test_scores, 
                              labels=sel_model, title=f"{label} - Model AUC - Test")
        plt.savefig(f'img/model_roc_test_{label}.svg', bbox_inches='tight')
        plt.show()
    else:
        print(f"长度不匹配: y_test_sel[{label}] 和 pred_test_scores[0]")


In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA
for pred_score, label in zip(pred_scores, labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            okcomp.comp1.plot_DCA(pred_score[sel_model_idx][0][:, 1], np.array(y_train_sel[label]),
                                  title=f'Model {sm} DCA - Training Set')
            plt.savefig(f'img/model_{sm}_dca_train_{label}.svg', bbox_inches='tight')
            plt.show()

for pred_score, label in zip(pred_scores, labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            okcomp.comp1.plot_DCA(pred_score[sel_model_idx][1][:, 1], np.array(y_val_data[label]),
                                  title=f'Model {sm} DCA - Internal Testing Set')
            plt.savefig(f'img/model_{sm}_dca_val_{label}.svg', bbox_inches='tight')
            plt.show()

for pred_score, label in zip(pred_scores, labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            okcomp.comp1.plot_DCA(pred_score[sel_model_idx][2][:, 1], np.array(y_test_sel[label]),
                                  title=f'Model {sm} DCA - External Testing Set')
            plt.savefig(f'img/model_{sm}_dca_test_{label}.svg', bbox_inches='tight')
            plt.show()


In [ ]:
sel_model = model_names
c_matrix = {}

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            cm = okcomp.comp1.calc_confusion_matrix(predictions[idx][sel_model_idx][0], y_train_sel[label],
                                                    class_mapping={1:'1', 0:'0'}, num_classes=2)
            plt.figure(figsize=(5, 4))
            plt.title(f'Model: {sm} - Training Set')
            okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.3g')
            plt.savefig(f'img/model_{sm}_cm_train_{label}.svg', bbox_inches='tight')
            plt.show()

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            cm = okcomp.comp1.calc_confusion_matrix(predictions[idx][sel_model_idx][1], y_val_data[label],
                                                    class_mapping={1:'1', 0:'0'}, num_classes=2)
            plt.figure(figsize=(5, 4))
            plt.title(f'Model: {sm} - Internal Testing Set')
            okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.3g')
            plt.savefig(f'img/model_{sm}_cm_val_{label}.svg', bbox_inches='tight')
            plt.show()

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            cm = okcomp.comp1.calc_confusion_matrix(predictions[idx][sel_model_idx][2], y_test_sel[label],
                                                    class_mapping={1:'1', 0:'0'}, num_classes=2)
            plt.figure(figsize=(5, 4))
            plt.title(f'Model: {sm} - External Testing Set')
            okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.3g')
            plt.savefig(f'img/model_{sm}_cm_test_{label}.svg', bbox_inches='tight')
            plt.show()


In [ ]:
sel_model = model_names
c_matrix = {}

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):            
            okcomp.comp1.draw_predict_score(pred_scores[idx][sel_model_idx][0], y_train_sel[label])
            plt.title(f'{sm} Sample Predict Score - Training S')
            plt.legend(labels=["label=0","label=1"], loc="lower right") 
            plt.savefig(f'img/model_{sm}_sample_dis_train_{label}.svg', bbox_inches='tight')
            plt.show()

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):            
            okcomp.comp1.draw_predict_score(pred_scores[idx][sel_model_idx][1], y_val_data[label])
            plt.title(f'{sm} Sample Predict Score - Internal Testing Set')
            plt.legend(labels=["label=0","label=1"], loc="lower right") 
            plt.savefig(f'img/model_{sm}_sample_dis_val_{label}.svg', bbox_inches='tight')
            plt.show()

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):            
            okcomp.comp1.draw_predict_score(pred_scores[idx][sel_model_idx][2], y_test_sel[label])
            plt.title(f'{sm} Sample Predict Score - External Testing Set')
            plt.legend(labels=["label=0","label=1"], loc="lower right") 
            plt.savefig(f'img/model_{sm}_sample_dis_test_{label}.svg', bbox_inches='tight')
            plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd

os.makedirs('results', exist_ok=True)
sel_model = model_names

for idx, label in enumerate(labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            target = targets[idx][sel_model_idx]
            train_indexes = np.reshape(np.array(train_ids), (-1, 1)).astype(str)
            val_indexes = np.reshape(np.array(val_ids), (-1, 1)).astype(str) 
            test_indexes = np.reshape(np.array(test_ids), (-1, 1)).astype(str)
            y_train_pred_scores = target.predict_proba(X_train_sel)
            y_val_pred_scores = target.predict_proba(X_val_data)  
            y_test_pred_scores = target.predict_proba(X_test_sel)
            columns = ['ID'] + [f"{label}-{i}" for i in range(y_test_pred_scores.shape[1])]
            result_train = pd.DataFrame(np.concatenate([train_indexes, y_train_pred_scores], axis=1), columns=columns)
            result_train.to_csv(f'results/Rad_{sm}_train_{label}.csv', index=False)
            result_val = pd.DataFrame(np.concatenate([val_indexes, y_val_pred_scores], axis=1), columns=columns)
            result_val.to_csv(f'results/Rad_{sm}_val_{label}.csv', index=False)
            result_test = pd.DataFrame(np.concatenate([test_indexes, y_test_pred_scores], axis=1), columns=columns)
            result_test.to_csv(f'results/Rad_{sm}_test_{label}.csv', index=False)

label_data.to_csv('group.csv', index=False)
